In [1]:
import requests
from typing import List, Optional, cast
from pystac import Collection, MediaType
from pystac_client import Client, CollectionClient
from datetime import datetime
import xarray as xr

# xr.set_options(display_expand_attrs=False)

In [2]:
def list_found_elements(search_result):
    id = []
    coll = []
    for item in search_result.items(): #retrieves the result inside the catalogue.
        id.append(item.id)
        coll.append(item.collection_id)
    return id , coll

In [3]:
max_description_length = 100

eopf_stac_api_root_endpoint = "https://stac.core.eopf.eodc.eu/" #root starting point
client = Client.open(url=eopf_stac_api_root_endpoint)

In [4]:
print(
    "Connected to Catalog {id}: {description}".format(
        id=client.id,
        description=client.description
        if len(client.description) <= max_description_length
        else client.description[: max_description_length - 3] + "...",
    )
)

Connected to Catalog eopf-sample-service-stac-api: STAC catalog of the EOPF Sentinel Zarr Samples Service


In [5]:
innsbruck_s2 = client.search(
    bbox=(11.124756, 47.311058, # AOI extent
          11.459839,47.463624),
    collections= ['sentinel-2-l2a'], # interest Collection
    datetime='2020-05-01T00:00:00Z/2025-05-31T23:59:59.999999Z' # interest period
)

In [6]:
new_ins=list_found_elements(innsbruck_s2)
print('Retrieved Sentinel 2 L2A Items between 01-May-2020 and 31-May-2025 close to Innsbruck, Austria: ',len(new_ins[0]))

Retrieved Sentinel 2 L2A Items between 01-May-2020 and 31-May-2025 close to Innsbruck, Austria:  27


In [7]:
c_sentinel2 = client.get_collection('sentinel-2-l2a')
c_sentinel2_urls=[]
for x in range(len(new_ins[0])):
    c_sentinel2_urls.append(c_sentinel2.get_item(new_ins[0][x]).self_href)

In [8]:
c_sentinel2_ids=new_ins[0]

In [9]:
c_sentinel2_ids

['S2B_MSIL2A_20250530T101559_N0511_R065_T32TPT_20250530T130924',
 'S2A_MSIL2A_20250527T102041_N0511_R065_T32TPT_20250527T165916',
 'S2B_MSIL2A_20250527T100559_N0511_R022_T32TPT_20250527T155229',
 'S2C_MSIL2A_20250525T101621_N0511_R065_T32TPT_20250525T153015',
 'S2A_MSIL2A_20250524T100701_N0511_R022_T32TPT_20250524T121311',
 'S2C_MSIL2A_20250522T100611_N0511_R022_T32TPT_20250522T153214',
 'S2A_MSIL2A_20250517T101701_N0511_R065_T32TPT_20250517T120915',
 'S2B_MSIL2A_20250517T100559_N0511_R022_T32TPT_20250517T130149',
 'S2C_MSIL2A_20250515T101611_N0511_R065_T32TPT_20250515T154718',
 'S2A_MSIL2A_20250514T101041_N0511_R022_T32TPT_20250514T164611',
 'S2C_MSIL2A_20250512T100611_N0511_R022_T32TPT_20250512T173114',
 'S2B_MSIL2A_20250510T101559_N0511_R065_T32TPT_20250510T131257',
 'S2B_MSIL2A_20250510T101559_N0511_R065_T32TPT_20250510T123915',
 'S2A_MSIL2A_20250507T102041_N0511_R065_T32TPT_20250507T140612',
 'S2B_MSIL2A_20250507T100559_N0511_R022_T32TPT_20250507T125642',
 'S2C_MSIL2A_20250505T101

In [10]:
#Choosing the first item available to be opened:

item=c_sentinel2.get_item(id=c_sentinel2_ids[0])

In [11]:
for asset_name, asset in sorted(
    item.get_assets(media_type=MediaType.ZARR).items(), key=lambda item: item[1].href
):
    roles = asset.roles or []
    print(
        "Zarr asset {group_path} ({title}) has roles {roles}".format(
            group_path="".join(asset.href.split(".zarr")[-1:]) or "/",
            title=asset.title,
            roles=roles,
        )
    )
    # Identify the top-level Zarr group asset. This is what we will access with xarray.
    if "data" in roles and "metadata" in roles:
        top_level_zarr_group_asset = asset

Zarr asset / (EOPF Product) has roles ['data', 'metadata']
Zarr asset /conditions/mask/l2a_classification/r20m/scl (Scene classification map (SCL)) has roles ['data']
Zarr asset /measurements/reflectance/r10m (Surface Reflectance - 10m) has roles ['data', 'reflectance', 'dataset']
Zarr asset /measurements/reflectance/r10m/b02 (Blue (band 2) - 10m) has roles ['data', 'reflectance']
Zarr asset /measurements/reflectance/r10m/b03 (Green (band 3) - 10m) has roles ['data', 'reflectance']
Zarr asset /measurements/reflectance/r10m/b04 (Red (band 4) - 10m) has roles ['data', 'reflectance']
Zarr asset /measurements/reflectance/r10m/b08 (NIR 1 (band 8) - 10m) has roles ['data', 'reflectance']
Zarr asset /measurements/reflectance/r20m (Surface Reflectance - 20m) has roles ['data', 'reflectance', 'dataset']
Zarr asset /measurements/reflectance/r20m/b01 (Coastal aerosol (band 1) - 20m) has roles ['data', 'reflectance']
Zarr asset /measurements/reflectance/r20m/b05 (Red edge 1 (band 5) - 20m) has rol

In [12]:
assert (
    top_level_zarr_group_asset is not None
), "Unable to find top-level Zarr group asset"
print(
    "Asset {name} is the top-level Zarr group asset".format(
        name=top_level_zarr_group_asset.title
    )
)

Asset EOPF Product is the top-level Zarr group asset


In [14]:
# dt = xr.open_datatree(top_level_zarr_group_asset.href, **top_level_zarr_group_asset.extra_fields["xarray:open_datatree_kwargs"])
# dt = xr.open_datatree(
#     top_level_zarr_group_asset.href,
#     **top_level_zarr_group_asset.extra_fields["xarray:open_datatree_kwargs"]
# )
# dt = xr.open_datatree(top_level_zarr_group_asset.href, engine="eopf-zarr", op_mode="native", chunks={})
# dt

dt = xr.open_datatree(
    top_level_zarr_group_asset.href, engine="eopf-zarr", op_mode="native", chunks={})
for dt_group in sorted(dt.groups):
    print("DataTree group {group_name}".format(group_name=dt_group))

AttributeError: `np.PINF` was removed in the NumPy 2.0 release. Use `np.inf` instead.

In [ ]:
top_level_zarr_group_asset.get_absolute_href()

In [ ]:
xr.open_datatree()

In [16]:
item

<Item id=S2B_MSIL2A_20250530T101559_N0511_R065_T32TPT_20250530T130924>